In [34]:
# http://stackoverflow.com/questions/12141150/from-list-of-integers-get-number-closest-to-a-given-value

from bisect import bisect_left

def takeClosest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, float(myNumber))
    if pos == 0:
        return float(myList[0])
    if pos == len(myList):
        return float(myList[-1])
    before = float(myList[pos - 1])
    after = float(myList[pos])
    if after - float(myNumber) < float(myNumber) - before:
        return after
    else:
        return before

In [35]:
def bin_search_theta_to_index(crack_points, low, high, target):
    mid = (low + high) // 2
    if float(crack_points[mid][4]) > target:
        return bin_search_theta_to_index(crack_points, low, mid - 1, target)
    elif float(crack_points[mid][4]) < target:
        return bin_search_theta_to_index(crack_points, mid + 1, high, target)
    elif float(crack_points[mid][4]) == target:
        return mid
    else:
        raise Exception("Binary search should never miss in this case...")

In [36]:
def get_distance(x1, y1, x2, y2):
    return ((x1 - x2)**2 + (y1 - y2)**2)**.5

In [37]:
def step_right_until_dist(crack_points, theta, minimum_distance, thetas):
    temp_theta = takeClosest(thetas, theta)
    start_point_index = bin_search_theta_to_index(crack_points, 0, len(crack_points), temp_theta)
    start_point = crack_points[start_point_index]
    start_x = float(start_point[5])
    start_y = float(start_point[6])
#     start_z = float(start_point[7])
    
    current_point_index = start_point_index
    current_point = crack_points[current_point_index]
    current_distance = 0
    while (current_point_index > 0 and current_point_index < len(crack_points) - 1 and current_distance <= minimum_distance):
        current_point_index += 1
        current_point = crack_points[current_point_index]
        current_x = float(current_point[5])
        current_y = float(current_point[6])
#         current_z = float(current_point[7])
        current_distance = get_distance(start_x, start_y, current_x, current_y)
    
    return current_point

In [38]:
def step_left_until_dist(crack_points, theta, minimum_distance, thetas):
    temp_theta = takeClosest(thetas, theta)
    start_point_index = bin_search_theta_to_index(crack_points, 0, len(crack_points), temp_theta)
    start_point = crack_points[start_point_index]
    start_x = float(start_point[5])
    start_y = float(start_point[6])
#     start_z = float(start_point[7])
    
    current_point_index = start_point_index
    current_point = crack_points[current_point_index]
    current_distance = 0
    while (current_point_index > 0 and current_point_index < len(crack_points) and current_distance <= minimum_distance):
        current_point_index -= 1
        current_point = crack_points[current_point_index]
        current_x = float(current_point[5])
        current_y = float(current_point[6])
#         current_z = float(current_point[7])
        current_distance = get_distance(start_x, start_y, current_x, current_y)
    
    return current_point

In [39]:
# Calculates the Taylor Factor and write it to file

num_cracks = 9
points_by_crack_id = []
thetas_by_crack_id = []
minimum_distance = 10 / 2
# This reference point is from the 0-crack_front_growth_rates_1500ppcf_transformed.csv file for CF_id=0
reference_x = -1.9932
reference_y = 363.26

natee = 0

for i in range(num_cracks):
    points_by_crack_id.append([])
    thetas_by_crack_id.append([])
    
with open('../../data/8-previous-dadN.csv', 'r') as r:
    # Skip the header of the file
    r.readline()

    for line in r:
        # Read in the tokens
        tokens = line.split(',')
        crack_id = int(tokens[3])
        theta = float(tokens[4])
        on_crack_front = tokens[8] == 'True'
        if on_crack_front and crack_id > 0:
            points_by_crack_id[crack_id].append(tokens)
            thetas_by_crack_id[crack_id].append(float(tokens[4]))

with open('../../data/8-previous-dadN.csv', 'r') as r:
    with open('../../data/9-curvature-distance.csv', 'w') as w:
        # Write the file headers
        headers = ('dadN,beta,change_in_z,crack_id,theta,x,y,z,on_crack_front,grain_id,nearest_grain_boundary_x,nearest_grain_boundary_y,'
                   'nearest_grain_boundary_z,nearest_grain_boundary_id,distance_to_grain_boundary,'
                   'vector_to_grain_boundary_x,vector_to_grain_boundary_y,vector_to_grain_boundary_z,'
                   'magnitude_of_vector_to_grain_boundary,unit_vector_to_grain_boundary_x,unit_vector_to_grain_boundary_y,'
                   'unit_vector_to_grain_boundary_z,phi1,Phi,phi2,'
                   'nearest_grain_phi1,nearest_grain_Phi,nearest_grain_phi2,'
                   'misorientation,normal_x,normal_y,normal_z,raw_angle,scaled_angle,taylor_factor,previous_dadN,curvature_distance\n')
        w.write(headers)
        
        r.readline()
        
        for line in r:
            tokens = line.split(',')
        
            dadN = tokens[0]
            beta = tokens[1]
            change_in_z = tokens[2]
            crack_id = int(tokens[3])
            theta = float(tokens[4])

            x = float(tokens[5])
            y = float(tokens[6])
            z = float(tokens[7])
            on_crack_front = tokens[8] == 'True'
            grain_id = int(tokens[9])

            nearest_grain_boundary_x = tokens[10]
            nearest_grain_boundary_y = tokens[11]
            nearest_grain_boundary_z = tokens[12]
            nearest_grain_boundary_id = int(tokens[13])

            distance_to_grain_boundary = tokens[14]
            vector_to_grain_boundary_x = float(tokens[15])
            vector_to_grain_boundary_y = float(tokens[16])
            vector_to_grain_boundary_z = float(tokens[17])
            magnitude_of_vector_to_grain_boundary = tokens[18]
            unit_vector_to_grain_boundary_x = tokens[19]
            unit_vector_to_grain_boundary_y = tokens[20]
            unit_vector_to_grain_boundary_z = tokens[21]

            phi1 = tokens[22]
            Phi = tokens[23]
            phi2 = tokens[24]
            nearest_grain_phi1 = tokens[25]
            nearest_grain_Phi = tokens[26]
            nearest_grain_phi2 = tokens[27]
            misorientation = tokens[28]

            normal_x = tokens[29]
            normal_y = tokens[30]
            normal_z = tokens[31]
            raw_angle = tokens[32]
            scaled_angle = tokens[33]
            taylor_factor = tokens[34]
            previous_dadN = tokens[35].strip()
            
            if crack_id > 0:
                right_point = step_right_until_dist(points_by_crack_id[crack_id], theta, minimum_distance, thetas_by_crack_id[crack_id])
                right_point_x = float(right_point[5])
                right_point_y = float(right_point[6])

                left_point = step_left_until_dist(points_by_crack_id[crack_id], theta, minimum_distance, thetas_by_crack_id[crack_id])
                left_point_x = float(left_point[5])
                left_point_y = float(left_point[6])

                distance_to_right = get_distance(right_point_x, right_point_y, reference_x, reference_y)
                distance_to_left = get_distance(left_point_x, left_point_y, reference_x, reference_y)
                distance_to_middle = get_distance(x, y, reference_x, reference_y)
                avg_distance_to_sides = (distance_to_right + distance_to_left) / 2
                curvature_distance = distance_to_middle - avg_distance_to_sides

            else:
                curvature_distance = None
            # Write it to the file
#             if crack_id > 0 and on_crack_front:
            to_write = '{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'\
                                .format(dadN,beta,change_in_z,crack_id,theta,x,y,z,on_crack_front,
                                grain_id,nearest_grain_boundary_x,nearest_grain_boundary_y,
                                nearest_grain_boundary_z,nearest_grain_boundary_id,distance_to_grain_boundary,
                                vector_to_grain_boundary_x,vector_to_grain_boundary_y,vector_to_grain_boundary_z,
                                magnitude_of_vector_to_grain_boundary,unit_vector_to_grain_boundary_x,
                                unit_vector_to_grain_boundary_y, unit_vector_to_grain_boundary_z, phi1, Phi, phi2, 
                                nearest_grain_phi1, nearest_grain_Phi, nearest_grain_phi2, misorientation, 
                                normal_x, normal_y, normal_z, raw_angle, scaled_angle, taylor_factor, previous_dadN, curvature_distance)
            w.write(to_write)
print('done')

done


In [2]:
# x1 = -0.93115
# y1 = 332.85
# z1 = 306.54

# x2 = 0.2469
# y2 = 330.57
# z2 = 306.95

# dist = ((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)**.5
# print(dist)

2.5989039617692895
